In [29]:
# !pip install pyvi

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import re
from pyvi import ViTokenizer

In [31]:
# from google.colab import drive
# drive.mount('/content/drive')

In [32]:
# %cd /content/drive/My Drive/School/Năm 3/Data Analysis/Final
# %ls -la

In [33]:
data = pd.read_csv('../data/mobile_feedback/Train.csv')
df_test=pd.read_csv('../data/mobile_feedback/Test.csv')
data.head(5)

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      7786 non-null   int64 
 1   comment    7786 non-null   object
 2   n_star     7786 non-null   int64 
 3   date_time  7786 non-null   object
 4   label      7786 non-null   object
dtypes: int64(2), object(3)
memory usage: 304.3+ KB


## Xử lý ngôn ngữ tiếng Việt cho mỗi bình luận

In [35]:
def remove_punctuation(comment):
  # Create a translation table
  translator = str.maketrans('', '', string.punctuation)
  # Remove punctuation
  new_string = comment.translate(translator)
  # Remove redudant space and break sign
  new_string = re.sub('[\n ]+', ' ', new_string)
  # Remove emoji icon
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
  new_string = re.sub(emoji_pattern, '', new_string)

  return new_string

In [36]:
def stopwords_vi():
    with open('../data/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()
        words = [line.split('\n')[0] for line in lines]
    return words

In [37]:
def remove_stopword(comment):
  stop_words = stopwords_vi()
  filtered = [word for word in comment.split() if word not in stop_words]

  return ' '.join(filtered)

> Execute function for df_train

In [38]:
data['comment'] = data['comment'].apply(lambda x: x.lower())
data['comment'] = data['comment'].apply(remove_punctuation)
data['comment'] = data['comment'].apply(remove_stopword)

>Execute function for df_test


In [39]:
df_test['comment'] = df_test['comment'].apply(lambda x: x.lower())
df_test['comment'] =df_test['comment'].apply(remove_punctuation)
df_test['comment'] = df_test['comment'].apply(remove_stopword)

In [40]:
data['comment'][23]

'tất mọi thứ ổn hết màn hình đẹp máy chạy êm pin sạc nhanh màu mặt lưng đẹp tiết ở phần thiết kế chi loa ngoài đặt ở bên hông thay đặt ở cạnh tay đè loa sử dụng í lo chính bản thân đè quá làm loa hư thêm 1 nhỏ xíu xiu mới mua hôm giảm 500000đ nay xem tuột xuống đươc giảm hẳn 1000000đ tức ghê'

In [41]:
df_test['comment'][23]

'máy mượt dùng tốt pin ok chụp hình đẹp mới mua về thời gian dùng tiếp'

In [42]:
df_test['comment'] = df_test['comment'].apply(lambda x: ViTokenizer.tokenize(x))
data['comment'] = data['comment'].apply(lambda x: ViTokenizer.tokenize(x))

## Gán nhãn cho mỗi bình luận

In [43]:
data['positive_count'] = data['label'].apply(lambda x: x.count("Positive"))
data['neutral_count'] = data['label'].apply(lambda x: x.count("Neutral"))
data['negative_count'] = data['label'].apply(lambda x: x.count("Negative"))

In [44]:
df_test['positive_count'] = df_test['label'].apply(lambda x: x.count("Positive"))
df_test['neutral_count'] = df_test['label'].apply(lambda x: x.count("Neutral"))
df_test['negative_count'] =df_test['label'].apply(lambda x: x.count("Negative"))

In [45]:
def assign_label(row):
    if row['positive_count'] > row['neutral_count'] and row['positive_count'] > row['negative_count']:
        return 'Positive'
    elif row['negative_count'] > row['neutral_count'] and row['negative_count'] > row['positive_count']:
        return 'Negative'
    elif row['negative_count'] == row['neutral_count'] :
        return 'Negative'
    elif row['neutral_count'] == row ['positive_count']:
        return "Positive"
    else :
        return "Neutral"

In [46]:
data['label'] = data.apply(assign_label, axis=1)

In [47]:
df_test['label']=df_test.apply(assign_label,axis=1)

In [48]:
data.head()

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,2,0,1


In [49]:
data['date_time'].value_counts()

date_time
2 tuần trước    216
3 tuần trước    197
1 tuần trước    169
4 tuần trước     70
21/04/2020       45
               ... 
15/06/2018        1
13/10/2018        1
22/07/2019        1
1/5/2018          1
15/06/2017        1
Name: count, Length: 756, dtype: int64

In [50]:
df_test['date_time'].value_counts()

date_time
2 tuần trước    70
3 tuần trước    56
1 tuần trước    47
4 tuần trước    20
20/02/2020      13
                ..
5/5/2018         1
17/08/2018       1
11/8/2019        1
24/09/2019       1
13/05/2019       1
Name: count, Length: 554, dtype: int64

In [51]:
def replace_invalid_dates(temp):
    try:
        temp['date_time'] = pd.to_datetime(temp['date_time'], errors='coerce')
    except Exception as ex:
        pass

    temp['date_time'].fillna(value=temp['date_time'].mode()[0], inplace=True)
    
    return temp

data = replace_invalid_dates(data)
df_test=replace_invalid_dates(df_test)

C:\Users\hdang\AppData\Local\Temp\ipykernel_12384\329145655.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp['date_time'] = pd.to_datetime(temp['date_time'], errors='coerce')
C:\Users\hdang\AppData\Local\Temp\ipykernel_12384\329145655.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temp['date_time'].fillna(value=temp['date_time'].mode()[0], inplace=True)
C:\Users\hdang\AppData\

In [52]:
data.head()

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2020-04-21,Positive,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,2019-09-14,Neutral,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,2020-08-17,Negative,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,2020-02-29,Neutral,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,2020-04-06,Positive,2,0,1


In [53]:
print(data.shape)
print(data.columns)

(7786, 8)
Index(['index', 'comment', 'n_star', 'date_time', 'label', 'positive_count',
       'neutral_count', 'negative_count'],
      dtype='object')


In [54]:
print(df_test.shape)
print(df_test.columns)

(2224, 8)
Index(['index', 'comment', 'n_star', 'date_time', 'label', 'positive_count',
       'neutral_count', 'negative_count'],
      dtype='object')


In [55]:
data.head()

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2020-04-21,Positive,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,2019-09-14,Neutral,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,2020-08-17,Negative,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,2020-02-29,Neutral,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,2020-04-06,Positive,2,0,1


In [56]:
data.to_csv("../data/mobile_feedback/Cleaned_Train.csv")
df_test.to_csv("../data/mobile_feedback/Cleaned_Test.csv")